In [2]:
def as_list(x):
    if type(x) is list:
        return x
    else:
        return [x]

In [3]:
from riotwatcher import LolWatcher, ApiError
import pandas as pd
import numpy as np

# global variables
api_key = 'RGAPI-d7d4df0e-05b7-4541-b09c-4daacb8dda9f'
watcher = LolWatcher(api_key)
my_region = 'euw1'


In [4]:
# fetch matchlist by puuid
me = watcher.summoner.by_name(my_region, 'Kelody')
# me = watcher.summoner.by_name(my_region, 'daimox')
my_matches = watcher.match.matchlist_by_puuid("EUROPE", me['puuid'])
matches = []
matchTimeLine = []
participants = []


In [5]:
# fetch game, participant and match timeline and transform into dataframe
for matchId in my_matches:
    match_detail = watcher.match.by_id("EUROPE", matchId)
    timeline = watcher.match.timeline_by_match("EUROPE", matchId)
    if match_detail["info"]["mapId"] in [1, 2, 11]:
        match = {}
        match["matchId"] = matchId
        match["gameDuration"] = match_detail["info"]["gameDuration"]
        match["gameType"] = match_detail["info"]["gameType"]
        match["gameVersion"] = match_detail["info"]["gameVersion"]
        matches.append(match)
        for index, row in enumerate(match_detail["info"]["participants"]):
            participants_row = {}
            participants_row["matchId"] = matchId
            participants_row["participant"] = index + 1
            participants_row["summonerName"] = row["summonerName"]
            participants_row["teamPosition"] = row["teamPosition"]
            participants_row["championName"] = row["championName"]
            participants_row["kda"] = f"{row['kills']}/{row['deaths']}/{row['assists']}"
            participants_row["champLevel"] = row["champLevel"]
            participants_row["totalDamageToChampions"] = row[
                "totalDamageDealtToChampions"
            ]
            participants_row["totalDamageTaken"] = row["totalDamageTaken"]
            participants_row["totalHeal"] = row["totalHeal"]
            participants_row["totalMinionsKilled"] = row["totalMinionsKilled"]
            participants_row["goldEarned"] = row["goldEarned"]
            participants_row["summoner1"] = row["summoner1Id"]
            participants_row["summoner2"] = row["summoner2Id"]
            participants_row["item0"] = row["item0"]
            participants_row["item1"] = row["item1"]
            participants_row["item2"] = row["item2"]
            participants_row["item3"] = row["item3"]
            participants_row["item4"] = row["item4"]
            participants_row["item5"] = row["item5"]
            participants_row["item6"] = row["item6"]
            participants_row["win"] = row["win"]
            participants.append(participants_row)

        for row in timeline["info"]["frames"]:
            # frameCounter = 1
            # frame = row["{:02d}".format(frameCounter)]
            # frameCounter += 1
            for frameRow in row["events"]:
                timeline_row = {}
                timeline_row["matchId"] = matchId
                eventCounter = 0
                # eventFrame = frameRow["{:02d}".format(eventCounter)]
                eventFrame = frameRow
                eventCounter += 1

                if eventFrame["type"] == "CHAMPION_KILL":
                    timeline_row["type"] = eventFrame["type"]
                    timeline_row["killerId"] = eventFrame["killerId"]
                    timeline_row["victimId"] = eventFrame["victimId"]
                    if "assistingParticipantIds" in eventFrame:
                        timeline_row["assistingParticipantIds"] = eventFrame[
                            "assistingParticipantIds"
                        ]
                    if "bounty" in eventFrame:
                        timeline_row["bounty"] = eventFrame["bounty"]
                    timeline_row["positionX"] = eventFrame["position"]["x"]
                    timeline_row["positionY"] = eventFrame["position"]["y"]
                    timeline_row["timestamp"] = eventFrame["timestamp"]
                    matchTimeLine.append(timeline_row)
                elif eventFrame["type"] == "BUILDING_KILL":
                    timeline_row["type"] = eventFrame["type"]
                    timeline_row["killerId"] = eventFrame["killerId"]
                    timeline_row["teamId"] = eventFrame["teamId"]
                    timeline_row["buildingType"] = eventFrame["buildingType"]
                    timeline_row["laneType"] = eventFrame["laneType"]
                    timeline_row["positionX"] = eventFrame["position"]["x"]
                    timeline_row["positionY"] = eventFrame["position"]["y"]
                    timeline_row["timestamp"] = eventFrame["timestamp"]
                    matchTimeLine.append(timeline_row)
                elif eventFrame["type"] == "ITEM_PURCHASED":
                    timeline_row["type"] = eventFrame["type"]
                    timeline_row["participantId"] = eventFrame["participantId"]
                    timeline_row["itemId"] = eventFrame["itemId"]
                    timeline_row["timestamp"] = eventFrame["timestamp"]
                    matchTimeLine.append(timeline_row)
                elif eventFrame["type"] == "WARD_PLACED":
                    timeline_row["type"] = eventFrame["type"]
                    timeline_row["wardType"] = eventFrame["wardType"]
                    timeline_row["creatorId"] = eventFrame["creatorId"]
                    timeline_row["timestamp"] = eventFrame["timestamp"]
                    matchTimeLine.append(timeline_row)
                elif eventFrame["type"] == "WARD_KILL":
                    timeline_row["type"] = eventFrame["type"]
                    timeline_row["wardType"] = eventFrame["wardType"]
                    timeline_row["killerId"] = eventFrame["killerId"]
                    timeline_row["timestamp"] = eventFrame["timestamp"]
                    matchTimeLine.append(timeline_row)
                elif eventFrame["type"] == "LEVEL_UP":
                    timeline_row["type"] = eventFrame["type"]
                    timeline_row["participantId"] = eventFrame["participantId"]
                    timeline_row["level"] = eventFrame["level"]
                    timeline_row["timestamp"] = eventFrame["timestamp"]
                    matchTimeLine.append(timeline_row)
                elif eventFrame["type"] == "ELITE_MONSTER_KILL":
                    timeline_row["type"] = eventFrame["type"]
                    timeline_row["killerId"] = eventFrame["killerId"]
                    timeline_row["teamId"] = eventFrame["killerTeamId"]
                    if "assistingParticipantIds" in eventFrame:
                        timeline_row["assistingParticipantIds"] = eventFrame[
                            "assistingParticipantIds"
                        ]
                    timeline_row["positionX"] = eventFrame["position"]["x"]
                    timeline_row["positionY"] = eventFrame["position"]["y"]
                    timeline_row["timestamp"] = eventFrame["timestamp"]
                    timeline_row["monsterType"] = eventFrame["monsterType"]
                    matchTimeLine.append(timeline_row)


participantsDf = pd.DataFrame(participants)
matchesDf = pd.DataFrame(matches)
matchTimelineDf = pd.DataFrame(matchTimeLine)
matchTimelineDf["timestampMIN"] = round(matchTimelineDf["timestamp"] / 60000, 0)


In [6]:
match_detail["info"]["mapId"]

12

In [7]:
participantsDf

,matchId,participant,summonerName,teamPosition,championName,kda,champLevel,totalDamageToChampions,totalDamageTaken,totalHeal,...,summoner1,summoner2,item0,item1,item2,item3,item4,item5,item6,win
0,EUW1_5803852070,1,Enkilzahar,TOP,Gnar,5/13/13,15,30449,42011,4066,...,12,4,6632,3053,3181,3047,0,0,3340,False
1,EUW1_5803852070,2,Lattardays,JUNGLE,Kindred,7/14/13,14,26336,38369,18130,...,4,11,6672,2031,6676,3036,0,0,3340,False
2,EUW1_5803852070,3,Shaeh,MIDDLE,Swain,9/13/13,15,39760,40823,9650,...,4,14,1056,6653,3191,3111,4637,3916,3340,False
3,EUW1_5803852070,4,Lilwai,BOTTOM,MissFortune,16/6/14,15,41658,18215,2130,...,7,4,6692,3042,6676,3036,3111,3134,3363,False
4,EUW1_5803852070,5,Mostrana,UTILITY,Leona,1/6/26,14,11301,25417,2188,...,14,4,3857,3001,3075,0,3111,2055,3364,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105,EUW1_5747855273,6,Junipie,TOP,Illaoi,3/3/1,15,9241,10082,1507,...,4,12,6632,3047,3133,3181,1031,1037,3340,False
106,EUW1_5747855273,7,NightWõlf,JUNGLE,Viego,2/4/3,12,8777,19859,6826,...,4,11,3047,2031,6632,1053,1037,1042,3340,False
107,EUW1_5747855273,8,Kelody,MIDDLE,Ahri,0/9/1,10,8515,13651,1479,...,4,14,1056,6655,3020,1052,0,0,3340,False
108,EUW1_5747855273,9,Sir Nurum,BOTTOM,Xayah,2/9/1,10,8737,14138,865,...,7,4,1018,0,6671,3006,1037,3134,3340,False


In [8]:
matchTimelineDf["assistingParticipantIds"] = [ [] if x is np.NaN else x for x in matchTimelineDf["assistingParticipantIds"]]

In [9]:
def getSummonerName(row, type):
    matchId = row["matchId"]
    if type == "participant":
        if pd.notna(row["participantId"]):
            participantId = row["participantId"]
            #print(participantsDf.loc[(participantsDf["participant"] == participantId) & (participantsDf["matchId"] == matchId)]["summonerName"].values)
            return participantsDf.loc[(participantsDf["participant"] == participantId) & (participantsDf["matchId"] == matchId)]["summonerName"].values[0]
        else:
            return float('NaN')
    elif type == "killer":
        if pd.notna(row["killerId"]):
            killerId = int(row["killerId"])
            try:
                return participantsDf.loc[(participantsDf["participant"] == killerId) & (participantsDf["matchId"] == matchId)]["summonerName"].values[0]
            except:
                return float('NaN')
        else:
            return float('NaN')
    elif type == "victim":
        if pd.notna(row["victimId"]):
            victimId = int(row["victimId"])
            try:
                return participantsDf.loc[(participantsDf["participant"] == victimId) & (participantsDf["matchId"] == matchId)]["summonerName"].values[0]
            except:
                return float('NaN')
        else:
            return float('NaN')
    elif type == "assist":
        pass
        #todo

def getChampionName(row, type):
    matchId = row["matchId"]
    if type == "participant":
        if pd.notna(row["participantId"]):
            participantId = row["participantId"]
            #print(participantsDf.loc[(participantsDf["participant"] == participantId) & (participantsDf["matchId"] == matchId)]["summonerName"].values)
            return participantsDf.loc[(participantsDf["participant"] == participantId) & (participantsDf["matchId"] == matchId)]["championName"].values[0]
        else:
            return float('NaN')
    elif type == "killer":
        if pd.notna(row["killerId"]):
            killerId = row["killerId"]
            try:
                return participantsDf.loc[(participantsDf["participant"] == killerId) & (participantsDf["matchId"] == matchId)]["championName"].values[0]
            except:
                return float('NaN')
        else:
            return float('NaN')
    elif type == "victim":
        if pd.notna(row["victimId"]):
            victimId = row["victimId"]
            try:
                return participantsDf.loc[(participantsDf["participant"] == victimId) & (participantsDf["matchId"] == matchId)]["championName"].values[0]
            except:
                return float('NaN')
        else:
            return float('NaN')
    elif type == "assist":
        pass
        #todo

def checkAssist(row, name):
    matchId = row["matchId"]
    if row["assistingParticipantIds"]:
        assistingParticipants = row["assistingParticipantIds"]
        for i in range(len(assistingParticipants)):
            if participantsDf.loc[(participantsDf["participant"] == assistingParticipants[i]) & (participantsDf["matchId"] == matchId)]["summonerName"].values[0] == name:
                return True
    return False


In [10]:
# matchTimelineDf.apply(lambda row: getSummonerName(row, "participant"), axis=1)
# matchTimelineDf.apply(lambda row: getSummonerName(row, "killer"), axis=1)
# matchTimelineDf.apply(lambda row: getSummonerName(row, "victim"), axis=1)
matchTimelineDf.apply(lambda row: checkAssist(row, "Kelody"), axis = 1)

0       False
1       False
2       False
3       False
4       False
        ...  
7203    False
7204    False
7205    False
7206    False
7207    False
Length: 7208, dtype: bool

In [11]:
matchTimelineDf["participantName"] = matchTimelineDf.apply(lambda row: getSummonerName(row, "participant"), axis=1)
matchTimelineDf["participantChampion"] = matchTimelineDf.apply(lambda row: getChampionName(row, "participant"), axis=1)
matchTimelineDf["killerName"] = matchTimelineDf.apply(lambda row: getSummonerName(row, "killer"), axis=1)
matchTimelineDf["killerChampion"] = matchTimelineDf.apply(lambda row: getChampionName(row, "killer"), axis=1)
matchTimelineDf["victimName"] = matchTimelineDf.apply(lambda row: getSummonerName(row, "victim"), axis=1)
matchTimelineDf["victimChampion"] = matchTimelineDf.apply(lambda row: getChampionName(row, "victim"), axis=1)
matchTimelineDf["kelodyAssist"] = matchTimelineDf.apply(lambda row: checkAssist(row, "Kelody"), axis = 1)


In [12]:
matchTimelineDf

,matchId,type,participantId,itemId,timestamp,killerId,victimId,assistingParticipantIds,bounty,positionX,...,buildingType,laneType,timestampMIN,participantName,participantChampion,killerName,killerChampion,victimName,victimChampion,kelodyAssist
0,EUW1_5803852070,ITEM_PURCHASED,4.0,1036.0,3739,NaN,NaN,[],NaN,NaN,...,NaN,NaN,0.0,Lilwai,MissFortune,NaN,NaN,NaN,NaN,False
1,EUW1_5803852070,ITEM_PURCHASED,5.0,3854.0,4003,NaN,NaN,[],NaN,NaN,...,NaN,NaN,0.0,Mostrana,Leona,NaN,NaN,NaN,NaN,False
2,EUW1_5803852070,ITEM_PURCHASED,5.0,2003.0,4003,NaN,NaN,[],NaN,NaN,...,NaN,NaN,0.0,Mostrana,Leona,NaN,NaN,NaN,NaN,False
3,EUW1_5803852070,ITEM_PURCHASED,5.0,2003.0,4003,NaN,NaN,[],NaN,NaN,...,NaN,NaN,0.0,Mostrana,Leona,NaN,NaN,NaN,NaN,False
4,EUW1_5803852070,ITEM_PURCHASED,4.0,2003.0,4367,NaN,NaN,[],NaN,NaN,...,NaN,NaN,0.0,Lilwai,MissFortune,NaN,NaN,NaN,NaN,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7203,EUW1_5747855273,WARD_KILL,NaN,NaN,1321811,4.0,NaN,[],NaN,NaN,...,NaN,NaN,22.0,NaN,NaN,ROHNEN1,Caitlyn,NaN,NaN,False
7204,EUW1_5747855273,CHAMPION_KILL,NaN,NaN,1323242,5.0,9.0,[2],274.0,7462.0,...,NaN,NaN,22.0,NaN,NaN,KG Maisen,Pyke,Sir Nurum,Xayah,False
7205,EUW1_5747855273,LEVEL_UP,5.0,NaN,1323242,NaN,NaN,[],NaN,NaN,...,NaN,NaN,22.0,KG Maisen,Pyke,NaN,NaN,NaN,NaN,False
7206,EUW1_5747855273,WARD_PLACED,NaN,NaN,1323676,NaN,NaN,[],NaN,NaN,...,NaN,NaN,22.0,NaN,NaN,NaN,NaN,NaN,NaN,False


In [13]:
kelody = participantsDf[participantsDf["summonerName"] == "Kelody"][["matchId", "participant"]]
kelody

,matchId,participant
6,EUW1_5803852070,7
16,EUW1_5803774910,7
21,EUW1_5803660385,2
31,EUW1_5803299160,2
41,EUW1_5803373057,2
51,EUW1_5803198068,2
66,EUW1_5803065394,7
76,EUW1_5802442397,7
87,EUW1_5802111424,8
97,EUW1_5747889998,8


In [14]:
kelodyEvents = []
#TODO bool if kelody was assist or not
# (matchTimelineDf["assistingParticipantIds"] == kelody["participant"].values[0]
for index, match in kelody.iterrows():
    game = []
    assist = matchTimelineDf[(matchTimelineDf["matchId"] == match["matchId"]) & matchTimelineDf["assistingParticipantIds"].notna() & (matchTimelineDf["creatorId"] != match["participant"]) & (matchTimelineDf["killerId"] != match["participant"]) & (matchTimelineDf["victimId"] != match["participant"])]
    participantList = as_list(match["participant"])
    game = matchTimelineDf.loc[((matchTimelineDf["matchId"] == match["matchId"]) & ((matchTimelineDf["participantId"] == (match["participant"])) | 
    (matchTimelineDf["creatorId"] == match["participant"]) |
    (matchTimelineDf["killerId"] == match["participant"]) |
    (matchTimelineDf["victimId"] == match["participant"]) |
    (assist["assistingParticipantIds"].apply(lambda x: any(i in x for i in participantList)))))]
    kelodyEvents.append(game)
kelodyEventsDf = pd.concat(kelodyEvents, ignore_index=True)
kelodyEventsDf

,matchId,type,participantId,itemId,timestamp,killerId,victimId,assistingParticipantIds,bounty,positionX,...,buildingType,laneType,timestampMIN,participantName,participantChampion,killerName,killerChampion,victimName,victimChampion,kelodyAssist
0,EUW1_5803852070,ITEM_PURCHASED,7.0,1035.0,13189,NaN,NaN,[],NaN,NaN,...,NaN,NaN,0.0,Kelody,Mordekaiser,NaN,NaN,NaN,NaN,False
1,EUW1_5803852070,ITEM_PURCHASED,7.0,2031.0,14214,NaN,NaN,[],NaN,NaN,...,NaN,NaN,0.0,Kelody,Mordekaiser,NaN,NaN,NaN,NaN,False
2,EUW1_5803852070,ITEM_PURCHASED,7.0,3340.0,14478,NaN,NaN,[],NaN,NaN,...,NaN,NaN,0.0,Kelody,Mordekaiser,NaN,NaN,NaN,NaN,False
3,EUW1_5803852070,WARD_PLACED,NaN,NaN,46191,NaN,NaN,[],NaN,NaN,...,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,False
4,EUW1_5803852070,LEVEL_UP,7.0,NaN,109436,NaN,NaN,[],NaN,NaN,...,NaN,NaN,2.0,Kelody,Mordekaiser,NaN,NaN,NaN,NaN,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
953,EUW1_5747855273,LEVEL_UP,8.0,NaN,1154602,NaN,NaN,[],NaN,NaN,...,NaN,NaN,19.0,Kelody,Ahri,NaN,NaN,NaN,NaN,False
954,EUW1_5747855273,ITEM_PURCHASED,8.0,6655.0,1168231,NaN,NaN,[],NaN,NaN,...,NaN,NaN,19.0,Kelody,Ahri,NaN,NaN,NaN,NaN,False
955,EUW1_5747855273,CHAMPION_KILL,NaN,NaN,1256497,2.0,8.0,[],112.0,10030.0,...,NaN,NaN,21.0,NaN,NaN,Takeasy,Evelynn,Kelody,Ahri,False
956,EUW1_5747855273,ITEM_PURCHASED,8.0,1052.0,1282981,NaN,NaN,[],NaN,NaN,...,NaN,NaN,21.0,Kelody,Ahri,NaN,NaN,NaN,NaN,False


In [15]:
kelodyEventsDf[kelodyEventsDf["kelodyAssist"] == True]

,matchId,type,participantId,itemId,timestamp,killerId,victimId,assistingParticipantIds,bounty,positionX,...,buildingType,laneType,timestampMIN,participantName,participantChampion,killerName,killerChampion,victimName,victimChampion,kelodyAssist
37,EUW1_5803852070,CHAMPION_KILL,NaN,NaN,763680,6.0,3.0,[7],300.0,1417.0,...,NaN,NaN,13.0,NaN,NaN,five night fredy,Fiora,Shaeh,Swain,True
47,EUW1_5803852070,CHAMPION_KILL,NaN,NaN,973386,6.0,1.0,[7],300.0,5811.0,...,NaN,NaN,16.0,NaN,NaN,five night fredy,Fiora,Enkilzahar,Gnar,True
54,EUW1_5803852070,CHAMPION_KILL,NaN,NaN,1114455,9.0,1.0,"[6, 7, 8, 10]",300.0,7431.0,...,NaN,NaN,19.0,NaN,NaN,Rayzam,Samira,Enkilzahar,Gnar,True
55,EUW1_5803852070,CHAMPION_KILL,NaN,NaN,1114686,6.0,3.0,"[7, 8, 9, 10]",274.0,7549.0,...,NaN,NaN,19.0,NaN,NaN,five night fredy,Fiora,Shaeh,Swain,True
67,EUW1_5803852070,CHAMPION_KILL,NaN,NaN,1347924,9.0,5.0,"[7, 8, 10]",300.0,2706.0,...,NaN,NaN,22.0,NaN,NaN,Rayzam,Samira,Mostrana,Leona,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
878,EUW1_5747889998,CHAMPION_KILL,NaN,NaN,919417,7.0,4.0,"[8, 10]",300.0,10436.0,...,NaN,NaN,15.0,NaN,NaN,NightWõlf,Karthus,GranpaLongShlong,MissFortune,True
879,EUW1_5747889998,ELITE_MONSTER_KILL,NaN,NaN,934511,7.0,NaN,"[8, 10]",NaN,9866.0,...,NaN,NaN,16.0,NaN,NaN,NightWõlf,Karthus,NaN,NaN,True
880,EUW1_5747889998,CHAMPION_KILL,NaN,NaN,974344,7.0,1.0,[8],300.0,6768.0,...,NaN,NaN,16.0,NaN,NaN,NightWõlf,Karthus,TrouserLeviathan,MonkeyKing,True
896,EUW1_5747889998,CHAMPION_KILL,NaN,NaN,1228414,10.0,4.0,"[7, 8, 9]",300.0,6232.0,...,NaN,NaN,20.0,NaN,NaN,Fegel,Braum,GranpaLongShlong,MissFortune,True


Check how often a Player died in Early Game (pre 10 mins)

In [16]:
sumOfDeaths = []
for index, match in kelody.iterrows():
     deaths = len(kelodyEventsDf[(kelodyEventsDf["matchId"] == match["matchId"]) & (kelodyEventsDf["victimId"] == match["participant"]) & (kelodyEventsDf["timestamp"] <= 600000)].index)
     sumOfDeaths.insert(len(sumOfDeaths), deaths)
kelody["deathsPreMin10"] = sumOfDeaths
kelody

,matchId,participant,deathsPreMin10
6,EUW1_5803852070,7,1
16,EUW1_5803774910,7,1
21,EUW1_5803660385,2,2
31,EUW1_5803299160,2,1
41,EUW1_5803373057,2,3
51,EUW1_5803198068,2,1
66,EUW1_5803065394,7,1
76,EUW1_5802442397,7,3
87,EUW1_5802111424,8,0
97,EUW1_5747889998,8,0


#### TODO
##### Features:
- <s>Died from ganks in laning phase pre 15 min (check ID from enemy Jungler)</s>

- Died with ... Gold

- <s>count 1v1 kill/deaths</s>  
  
- Good roaming (assists pre 15 min)

- Warding (check ward placements pre 15 min) maybe in combination with ganks

- CS per min (if data available)

- KDA (kills and assists divided by deaths) als Output KDA String aus participantDf

- <s>Tower Kills/Assists</s> 

- <s>Objective Kills/Assists (Baron, Dragon, Herald)</s> 



##### Features on the tableau visualization:

- <s>Timeline</s>

- <s>Champion Icons</s>

- <s>Damage Dealt Bar Chart (participantDf)</s>

- Game win (participantDf)



Check 1v1

In [17]:
kelodyEventsDf[kelodyEventsDf["kelodyAssist"] == False]

def check1v1(row, name):
    if not row["assistingParticipantIds"] and (row["victimName"] == name or row["killerName"] == name) and (row["type"] == "CHAMPION_KILL"):
        return True
    return False

kelodyEventsDf["1v1"] = kelodyEventsDf.apply(lambda row: check1v1(row, "Kelody"), axis = 1)
kelodyEventsDf[kelodyEventsDf["1v1"] == True]

,matchId,type,participantId,itemId,timestamp,killerId,victimId,assistingParticipantIds,bounty,positionX,...,laneType,timestampMIN,participantName,participantChampion,killerName,killerChampion,victimName,victimChampion,kelodyAssist,1v1
17,EUW1_5803852070,CHAMPION_KILL,NaN,NaN,417287,7.0,2.0,[],300.0,10116.0,...,NaN,7.0,NaN,NaN,Kelody,Mordekaiser,Lattardays,Kindred,False,True
28,EUW1_5803852070,CHAMPION_KILL,NaN,NaN,613816,7.0,3.0,[],300.0,7646.0,...,NaN,10.0,NaN,NaN,Kelody,Mordekaiser,Shaeh,Swain,False,True
29,EUW1_5803852070,CHAMPION_KILL,NaN,NaN,634039,7.0,1.0,[],175.0,5035.0,...,NaN,11.0,NaN,NaN,Kelody,Mordekaiser,Enkilzahar,Gnar,False,True
43,EUW1_5803852070,CHAMPION_KILL,NaN,NaN,869501,7.0,2.0,[],300.0,7661.0,...,NaN,14.0,NaN,NaN,Kelody,Mordekaiser,Lattardays,Kindred,False,True
60,EUW1_5803852070,CHAMPION_KILL,NaN,NaN,1213080,7.0,1.0,[],300.0,8740.0,...,NaN,20.0,NaN,NaN,Kelody,Mordekaiser,Enkilzahar,Gnar,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
938,EUW1_5747855273,CHAMPION_KILL,NaN,NaN,512048,3.0,8.0,[],274.0,8124.0,...,NaN,9.0,NaN,NaN,r3dcactus,Neeko,Kelody,Ahri,False,True
941,EUW1_5747855273,CHAMPION_KILL,NaN,NaN,652234,3.0,8.0,[],219.0,7360.0,...,NaN,11.0,NaN,NaN,r3dcactus,Neeko,Kelody,Ahri,False,True
949,EUW1_5747855273,CHAMPION_KILL,NaN,NaN,1017707,5.0,8.0,[],140.0,8126.0,...,NaN,17.0,NaN,NaN,KG Maisen,Pyke,Kelody,Ahri,False,True
955,EUW1_5747855273,CHAMPION_KILL,NaN,NaN,1256497,2.0,8.0,[],112.0,10030.0,...,NaN,21.0,NaN,NaN,Takeasy,Evelynn,Kelody,Ahri,False,True


In [18]:
def count1v1(row, name, killer):
  if killer:
    return len(kelodyEventsDf[(kelodyEventsDf["matchId"] == row["matchId"]) & (kelodyEventsDf["1v1"] == True) & (kelodyEventsDf["killerName"] == name)])
  else: 
    return len(kelodyEventsDf[(kelodyEventsDf["matchId"] == row["matchId"]) & (kelodyEventsDf["1v1"] == True) & (kelodyEventsDf["victimName"] == name)])

kelody["1v1Won"] = kelody.apply(lambda row: count1v1(row, "Kelody", True), axis=1)
kelody["1v1Lost"] = kelody.apply(lambda row: count1v1(row, "Kelody", False), axis=1)
kelody

,matchId,participant,deathsPreMin10,1v1Won,1v1Lost
6,EUW1_5803852070,7,1,11,0
16,EUW1_5803774910,7,1,7,1
21,EUW1_5803660385,2,2,5,1
31,EUW1_5803299160,2,1,2,0
41,EUW1_5803373057,2,3,1,1
51,EUW1_5803198068,2,1,3,1
66,EUW1_5803065394,7,1,3,5
76,EUW1_5802442397,7,3,7,2
87,EUW1_5802111424,8,0,3,8
97,EUW1_5747889998,8,0,5,1


In [19]:
def countTurretKills(row, name):
    return len(kelodyEventsDf[(kelodyEventsDf["matchId"] == row["matchId"]) & (kelodyEventsDf["type"] == "BUILDING_KILL") & (kelodyEventsDf["killerName"] == name) & (kelodyEventsDf["1v1"] == False)])

kelody["turretKills"] = kelody.apply(lambda row: countTurretKills(row, "Kelody"), axis=1)
kelody

,matchId,participant,deathsPreMin10,1v1Won,1v1Lost,turretKills
6,EUW1_5803852070,7,1,11,0,1
16,EUW1_5803774910,7,1,7,1,0
21,EUW1_5803660385,2,2,5,1,0
31,EUW1_5803299160,2,1,2,0,6
41,EUW1_5803373057,2,3,1,1,1
51,EUW1_5803198068,2,1,3,1,1
66,EUW1_5803065394,7,1,3,5,2
76,EUW1_5802442397,7,3,7,2,1
87,EUW1_5802111424,8,0,3,8,1
97,EUW1_5747889998,8,0,5,1,1


In [20]:
def counEliteMonsterKills(row, monsterName):
    return len(kelodyEventsDf[(kelodyEventsDf["matchId"] == row["matchId"]) & (kelodyEventsDf["type"] == "ELITE_MONSTER_KILL") & (kelodyEventsDf["monsterType"] == monsterName)])

kelody["dragonKills"] = kelody.apply(lambda row: counEliteMonsterKills(row, "DRAGON"), axis=1)
kelody["baronKills"] = kelody.apply(lambda row: counEliteMonsterKills(row, "BARON_NASHOR"), axis=1)
kelody["heraldKills"] = kelody.apply(lambda row: counEliteMonsterKills(row, "RIFTHERALD"), axis=1)

kelody

,matchId,participant,deathsPreMin10,1v1Won,1v1Lost,turretKills,dragonKills,baronKills,heraldKills
6,EUW1_5803852070,7,1,11,0,1,3,0,1
16,EUW1_5803774910,7,1,7,1,0,2,0,1
21,EUW1_5803660385,2,2,5,1,0,3,0,0
31,EUW1_5803299160,2,1,2,0,6,1,0,2
41,EUW1_5803373057,2,3,1,1,1,2,1,1
51,EUW1_5803198068,2,1,3,1,1,3,0,0
66,EUW1_5803065394,7,1,3,5,2,4,1,1
76,EUW1_5802442397,7,3,7,2,1,1,0,1
87,EUW1_5802111424,8,0,3,8,1,1,2,0
97,EUW1_5747889998,8,0,5,1,1,1,0,0


In [21]:
def deathToGank(row, name):
    if row["1v1"] == False and row["type"] == "CHAMPION_KILL" and row["victimName"] == name: 
        enemyJungler = participantsDf.loc[(row["matchId"] == participantsDf["matchId"]) & (participantsDf["teamPosition"] == "JUNGLE")]["summonerName"].values[0]
        if row["killerName"] == enemyJungler:
            return True
        assistingParticipants = row["assistingParticipantIds"]
        for i in range(len(assistingParticipants)):
            if participantsDf.loc[(participantsDf["participant"] == assistingParticipants[i]) & (participantsDf["matchId"] == row["matchId"])]["teamPosition"].values[0] == "JUNGLE":
                return True
    return False
kelodyEventsDf["deathToGank"] = kelodyEventsDf.apply(lambda row: deathToGank(row, "Kelody"), axis=1)

In [22]:
def countDeathsToGanks(row, name):
    return len(kelodyEventsDf[(kelodyEventsDf["matchId"] == row["matchId"]) & (kelodyEventsDf["deathToGank"] == True) & (kelodyEventsDf["victimName"] == name) & (kelodyEventsDf["timestamp"] <= 600000)])
kelody["deathsToGanks"] = kelody.apply(lambda row: countDeathsToGanks(row, "Kelody"), axis=1)

In [23]:
kelody

,matchId,participant,deathsPreMin10,1v1Won,1v1Lost,turretKills,dragonKills,baronKills,heraldKills,deathsToGanks
6,EUW1_5803852070,7,1,11,0,1,3,0,1,1
16,EUW1_5803774910,7,1,7,1,0,2,0,1,0
21,EUW1_5803660385,2,2,5,1,0,3,0,0,0
31,EUW1_5803299160,2,1,2,0,6,1,0,2,1
41,EUW1_5803373057,2,3,1,1,1,2,1,1,0
51,EUW1_5803198068,2,1,3,1,1,3,0,0,0
66,EUW1_5803065394,7,1,3,5,2,4,1,1,0
76,EUW1_5802442397,7,3,7,2,1,1,0,1,1
87,EUW1_5802111424,8,0,3,8,1,1,2,0,0
97,EUW1_5747889998,8,0,5,1,1,1,0,0,0


In [27]:
participantsDf


,matchId,participant,summonerName,teamPosition,championName,kda,champLevel,totalDamageToChampions,totalDamageTaken,totalHeal,...,summoner1,summoner2,item0,item1,item2,item3,item4,item5,item6,win
0,EUW1_5803852070,1,Enkilzahar,TOP,Gnar,5/13/13,15,30449,42011,4066,...,12,4,6632,3053,3181,3047,0,0,3340,False
1,EUW1_5803852070,2,Lattardays,JUNGLE,Kindred,7/14/13,14,26336,38369,18130,...,4,11,6672,2031,6676,3036,0,0,3340,False
2,EUW1_5803852070,3,Shaeh,MIDDLE,Swain,9/13/13,15,39760,40823,9650,...,4,14,1056,6653,3191,3111,4637,3916,3340,False
3,EUW1_5803852070,4,Lilwai,BOTTOM,MissFortune,16/6/14,15,41658,18215,2130,...,7,4,6692,3042,6676,3036,3111,3134,3363,False
4,EUW1_5803852070,5,Mostrana,UTILITY,Leona,1/6/26,14,11301,25417,2188,...,14,4,3857,3001,3075,0,3111,2055,3364,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105,EUW1_5747855273,6,Junipie,TOP,Illaoi,3/3/1,15,9241,10082,1507,...,4,12,6632,3047,3133,3181,1031,1037,3340,False
106,EUW1_5747855273,7,NightWõlf,JUNGLE,Viego,2/4/3,12,8777,19859,6826,...,4,11,3047,2031,6632,1053,1037,1042,3340,False
107,EUW1_5747855273,8,Kelody,MIDDLE,Ahri,0/9/1,10,8515,13651,1479,...,4,14,1056,6655,3020,1052,0,0,3340,False
108,EUW1_5747855273,9,Sir Nurum,BOTTOM,Xayah,2/9/1,10,8737,14138,865,...,7,4,1018,0,6671,3006,1037,3134,3340,False


In [32]:
def countWardsPlaced(row, name):
    participantId = participantsDf.loc[(participantsDf["matchId"] == row["matchId"]) & (participantsDf["summonerName"] == name)]["participant"].values[0]
    return len(kelodyEventsDf[(kelodyEventsDf["matchId"] == row["matchId"]) & (kelodyEventsDf["creatorId"] == participantId) & (kelodyEventsDf["type"] == "WARD_PLACED")])
kelody["wardsPlaced"] = kelody.apply(lambda row: countWardsPlaced(row, "Kelody"), axis=1)

In [33]:
kelody

,matchId,participant,deathsPreMin10,1v1Won,1v1Lost,turretKills,dragonKills,baronKills,heraldKills,deathsToGanks,wardsPlaced
6,EUW1_5803852070,7,1,11,0,1,3,0,1,1,5
16,EUW1_5803774910,7,1,7,1,0,2,0,1,0,7
21,EUW1_5803660385,2,2,5,1,0,3,0,0,0,4
31,EUW1_5803299160,2,1,2,0,6,1,0,2,1,11
41,EUW1_5803373057,2,3,1,1,1,2,1,1,0,6
51,EUW1_5803198068,2,1,3,1,1,3,0,0,0,5
66,EUW1_5803065394,7,1,3,5,2,4,1,1,0,6
76,EUW1_5802442397,7,3,7,2,1,1,0,1,1,8
87,EUW1_5802111424,8,0,3,8,1,1,2,0,0,24
97,EUW1_5747889998,8,0,5,1,1,1,0,0,0,15


In [ ]:
def countWardsPlacedBeforeGanked(row, name):
    return len(kelodyEventsDf[(kelodyEventsDf["matchId"] == row["matchId"]) & (kelodyEventsDf["deathToGank"] == True) & (kelodyEventsDf["victimName"] == name) & (kelodyEventsDf["timestamp"] <= 600000)])

    #TODO

In [26]:
participantsDf.to_json('participants.json', orient='records')
matchesDf.to_json('matches.json', orient='records')
kelodyEventsDf.to_json('kelodyEvents.json', orient='records')
kelody.to_json('kelodyStats.json', orient='records')
matchTimelineDf.to_json('matchTimeline.json', orient='records')
